In [1]:
import os
import json
import random
from pathlib import Path
from zipfile import ZipFile

In [2]:
import string
import re

In [5]:
!pip3 install textblob textblob_fr

  Using cached textblob_fr-0.2.0-py2.py3-none-any.whl (561 kB)


In [7]:
!pip3 install plotly

     |████████████████████████████████| 27.7 MB 37.2 MB/s eta 0:00:01
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


In [8]:
import pandas as pd 
from nltk.tokenize import sent_tokenize, word_tokenize

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

import plotly.graph_objects as go
import plotly.express as px

In [11]:
df = pd.read_csv('../../Resources/communes_commentaire.csv', delimiter=';'); df.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur
0,1004,1500,Ambérieu-en-Bugey,5.0,4.0,5.0,5.0,4.0,2,0,"Ville très agréable, tranquille et disposant d...",15/05/2021,INNO
1,1004,1500,Ambérieu-en-Bugey,4.0,4.0,4.0,4.0,3.0,0,1,La police et la gendarmerie peuvent se montrer...,03/03/2021,Jane
2,1004,1500,Ambérieu-en-Bugey,5.0,5.0,4.0,4.0,4.0,0,0,Ville ou sa ne vis pas beaucoup,04/11/2020,lilalou
3,1004,1500,Ambérieu-en-Bugey,2.0,4.0,2.0,4.0,2.0,0,0,C’est une belle ville mais peu d’activités (sp...,20/08/2019,Le tounsi
4,1004,1500,Ambérieu-en-Bugey,4.0,2.0,3.0,3.0,3.0,0,0,"Aspects positifs: la mixité sociale, cohabitat...",13/04/2018,Laetitialexandre


In [12]:
data = df.copy()
data.shape

(24344, 13)

In [13]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
3,Sécurité,2.805619
4,Éducation,2.805619
5,Sport / Loisir,2.805619
6,Environnement,2.805619
7,Vie pratique,2.805619
0,Insee,0.000000
1,Code_postal,0.000000
2,Ville,0.000000
8,pouce_positif,0.000000
9,pouce_negatif,0.000000


In [14]:
data = data.dropna()

In [15]:
data.shape

(23661, 13)

In [16]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
0,Insee,0.0
1,Code_postal,0.0
2,Ville,0.0
3,Sécurité,0.0
4,Éducation,0.0
5,Sport / Loisir,0.0
6,Environnement,0.0
7,Vie pratique,0.0
8,pouce_positif,0.0
9,pouce_negatif,0.0


In [17]:
data["commentaire"]

0        Ville très agréable, tranquille et disposant d...
1        La police et la gendarmerie peuvent se montrer...
2                         Ville ou sa ne vis pas beaucoup 
3        C’est une belle ville mais peu d’activités (sp...
4        Aspects positifs: la mixité sociale, cohabitat...
                               ...                        
24339    Ville qui est devenue sale. La gare n est pas ...
24340                        Qualité d’ enseignement zéro 
24341    Bien desservie pour aller à Paris. Nombreux tr...
24342    Proche des commodités de Paris limite de la ca...
24343                                               Pauvre
Name: commentaire, Length: 23661, dtype: object

In [18]:
data.iloc[:,10]

0        Ville très agréable, tranquille et disposant d...
1        La police et la gendarmerie peuvent se montrer...
2                         Ville ou sa ne vis pas beaucoup 
3        C’est une belle ville mais peu d’activités (sp...
4        Aspects positifs: la mixité sociale, cohabitat...
                               ...                        
24339    Ville qui est devenue sale. La gare n est pas ...
24340                        Qualité d’ enseignement zéro 
24341    Bien desservie pour aller à Paris. Nombreux tr...
24342    Proche des commodités de Paris limite de la ca...
24343                                               Pauvre
Name: commentaire, Length: 23661, dtype: object

In [19]:
data['Comment'] = data['commentaire'].apply(lambda f: f.lower())

In [20]:
AComment=[]
for comment in data["Comment"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",comment ).split():
        Word_Tok.append(word)
    AComment.append(Word_Tok)

In [21]:
data["Word_Tok"] = AComment
data.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur,Comment,Word_Tok
0,1004,1500,Ambérieu-en-Bugey,5.0,4.0,5.0,5.0,4.0,2,0,"Ville très agréable, tranquille et disposant d...",15/05/2021,INNO,"ville très agréable, tranquille et disposant d...","[ville, très, agréable, tranquille, et, dispos..."
1,1004,1500,Ambérieu-en-Bugey,4.0,4.0,4.0,4.0,3.0,0,1,La police et la gendarmerie peuvent se montrer...,03/03/2021,Jane,la police et la gendarmerie peuvent se montrer...,"[la, police, et, la, gendarmerie, peuvent, se,..."
2,1004,1500,Ambérieu-en-Bugey,5.0,5.0,4.0,4.0,4.0,0,0,Ville ou sa ne vis pas beaucoup,04/11/2020,lilalou,ville ou sa ne vis pas beaucoup,"[ville, ou, sa, ne, vis, pas, beaucoup]"
3,1004,1500,Ambérieu-en-Bugey,2.0,4.0,2.0,4.0,2.0,0,0,C’est une belle ville mais peu d’activités (sp...,20/08/2019,Le tounsi,c’est une belle ville mais peu d’activités (sp...,"[c, est, une, belle, ville, mais, peu, d, acti..."
4,1004,1500,Ambérieu-en-Bugey,4.0,2.0,3.0,3.0,3.0,0,0,"Aspects positifs: la mixité sociale, cohabitat...",13/04/2018,Laetitialexandre,"aspects positifs: la mixité sociale, cohabitat...","[aspects, positifs, la, mixité, sociale, cohab..."


In [22]:
stop_words=set(STOP_WORDS)

deselect_stop_words = ['n\'', 'ne','pas','plus','personne','aucun','ni','aucune','rien']
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
        continue

In [23]:
AllfilteredComment=[]
for comment in data["Word_Tok"]:
    filteredComment = [w for w in comment if not ((w in stop_words) or (len(w) == 1))]
    AllfilteredComment.append(' '.join(filteredComment))

In [24]:
data["CommentAferPreproc"]=AllfilteredComment
data.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur,Comment,Word_Tok,CommentAferPreproc
0,1004,1500,Ambérieu-en-Bugey,5.0,4.0,5.0,5.0,4.0,2,0,"Ville très agréable, tranquille et disposant d...",15/05/2021,INNO,"ville très agréable, tranquille et disposant d...","[ville, très, agréable, tranquille, et, dispos...",ville agréable tranquille disposant commodités...
1,1004,1500,Ambérieu-en-Bugey,4.0,4.0,4.0,4.0,3.0,0,1,La police et la gendarmerie peuvent se montrer...,03/03/2021,Jane,la police et la gendarmerie peuvent se montrer...,"[la, police, et, la, gendarmerie, peuvent, se,...",police gendarmerie montrer trop répressives in...
2,1004,1500,Ambérieu-en-Bugey,5.0,5.0,4.0,4.0,4.0,0,0,Ville ou sa ne vis pas beaucoup,04/11/2020,lilalou,ville ou sa ne vis pas beaucoup,"[ville, ou, sa, ne, vis, pas, beaucoup]",ville ne vis pas beaucoup
3,1004,1500,Ambérieu-en-Bugey,2.0,4.0,2.0,4.0,2.0,0,0,C’est une belle ville mais peu d’activités (sp...,20/08/2019,Le tounsi,c’est une belle ville mais peu d’activités (sp...,"[c, est, une, belle, ville, mais, peu, d, acti...",belle ville activités sportifs divertissante
4,1004,1500,Ambérieu-en-Bugey,4.0,2.0,3.0,3.0,3.0,0,0,"Aspects positifs: la mixité sociale, cohabitat...",13/04/2018,Laetitialexandre,"aspects positifs: la mixité sociale, cohabitat...","[aspects, positifs, la, mixité, sociale, cohab...",aspects positifs mixité sociale cohabitation h...


In [25]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [26]:
senti_list = []
for i in data["CommentAferPreproc"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positive')
    elif (vs < 0):
        senti_list.append('Negative')
    else:
        senti_list.append('Neutral')

> tb = textBlob.Blobber()

In [27]:
data["sentiment"] = senti_list
data.head()

,Insee,Code_postal,Ville,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,pouce_positif,pouce_negatif,commentaire,date,auteur,Comment,Word_Tok,CommentAferPreproc,sentiment
0,1004,1500,Ambérieu-en-Bugey,5.0,4.0,5.0,5.0,4.0,2,0,"Ville très agréable, tranquille et disposant d...",15/05/2021,INNO,"ville très agréable, tranquille et disposant d...","[ville, très, agréable, tranquille, et, dispos...",ville agréable tranquille disposant commodités...,Positive
1,1004,1500,Ambérieu-en-Bugey,4.0,4.0,4.0,4.0,3.0,0,1,La police et la gendarmerie peuvent se montrer...,03/03/2021,Jane,la police et la gendarmerie peuvent se montrer...,"[la, police, et, la, gendarmerie, peuvent, se,...",police gendarmerie montrer trop répressives in...,Negative
2,1004,1500,Ambérieu-en-Bugey,5.0,5.0,4.0,4.0,4.0,0,0,Ville ou sa ne vis pas beaucoup,04/11/2020,lilalou,ville ou sa ne vis pas beaucoup,"[ville, ou, sa, ne, vis, pas, beaucoup]",ville ne vis pas beaucoup,Negative
3,1004,1500,Ambérieu-en-Bugey,2.0,4.0,2.0,4.0,2.0,0,0,C’est une belle ville mais peu d’activités (sp...,20/08/2019,Le tounsi,c’est une belle ville mais peu d’activités (sp...,"[c, est, une, belle, ville, mais, peu, d, acti...",belle ville activités sportifs divertissante,Positive
4,1004,1500,Ambérieu-en-Bugey,4.0,2.0,3.0,3.0,3.0,0,0,"Aspects positifs: la mixité sociale, cohabitat...",13/04/2018,Laetitialexandre,"aspects positifs: la mixité sociale, cohabitat...","[aspects, positifs, la, mixité, sociale, cohab...",aspects positifs mixité sociale cohabitation h...,Negative
